In [1]:
import time
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import math

import sys
sys.path.append("../..")

In [2]:
from d2l import d2l_cn as d2l

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [5]:
num_hiddens = 256

In [6]:
rnn_layer = nn.RNN(input_size = vocab_size, hidden_size = num_hiddens)

In [7]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps,batch_size,vocab_size)
Y, state_new = rnn_layer(X,state)

In [8]:
print(Y.shape, len(state_new), state_new[0].shape)
#Y 是rnn的输出
#state_new是最新的隐藏层状态

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [9]:
class RNNModule(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModule,self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size,vocab_size)
        self.state = None
    
    def forward(self,inputs,state):
        #inputs (batch, seq_len)
        X = d2l.to_onehot(inputs,self.vocab_size)
        Y, self.state = self.rnn(torch.stack(X),state)
        output = self.dense(Y.view(-1,Y.shape[-1]))
        return output, self.state

In [10]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char, char_to_idx):
    state = None
    #output包含输入前缀
    output = [char_to_idx[prefix[0]]]
    
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]],device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple):#这种情况是LSTM网络
                state = (state[0].to(device),state[1].to(device))
            else:
                state = state.to(device)
        
        (Y,state) = model(X,state)
        if t<len(prefix) -1:
            #如果时间步还是在前缀里，直接拿前缀的字符
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
        
    return ''.join([idx_to_char[i] for i in output] )
                

In [11]:
model = RNNModule(rnn_layer,vocab_size).to(device)

In [12]:
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开葬背背背背背背背背背'

In [13]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    
    model.to(device)
    
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        
        data_iter = d2l. data_iter_consecutive(corpus_indices,batch_size,num_steps,device)#相邻采样
        
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state,tuple):#LSTM
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()#从计算图剥离
            
            (output, state) = model(X,state)#output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            l = loss(output,y.long())
            
            optimizer.zero_grad()
            #反向传播
            l.backward()
            d2l.grad_clipping(model.parameters(),clipping_theta,device)
            optimizer.step()
            #为啥要乘一下？
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
            
        try:
            perplexity = math.exp(l_sum/n)
        except OverflowError:
            perplexity = float('inf')
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [14]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.979730, time 2.95 sec
 - 分开 为什么这样子巷　 所有人们都很美主义 太彻底 分手当作我的开意  我们进武情在演下 一步两步三步四
 - 不分开 爱你的美不 你来难过 只是你们的爱情 再给我一种 所有回忆 我用第臂人开始使用第一人称在飘移青春 
epoch 100, perplexity 1.085944, time 3.45 sec
 - 分开 爱去的思念 不要有 才能够明白 我想想起 嘴角色刀调 你小命 是因为一点也许时还是不在远气　别会微
 - 不分开 为什么这样子我全拉着我 也还有种全神 我会学着我的方 这么简单你做不到 但们当朋友 泪是清晰的爱你
epoch 150, perplexity 1.044751, time 2.94 sec
 - 分开 为什么想 我开你的话 好像 认真的男人最美丽 风 是否在她里 里懂西七看到 到你的身影 街上的一切
 - 不分开 为什么证明被别人的手前 你用手牵手 你身为我手猜 所以你弃权到我对你的想 当我这第一个人的梦会绕过
epoch 200, perplexity 1.032920, time 3.01 sec
 - 分开 爱他的思是你不要让我也能不能不能太爱情 是因开没有办法 用来的方式 我们都还很年幼 而如今琴声幽幽
 - 不分开 爱你的爱情 就是我一直接 我要你离开 回忆很可爱没有祝福的话怎么每后每天都 我要我还要会抱 我说的
epoch 250, perplexity 1.032565, time 3.04 sec
 - 分开 爱恨的对不 就算是因为在乎再不 用力在我中给的承诺 你说我若一个人会比较自由 我不懂你说什么 反正
 - 不分开 爱你的嘴角 微眼中撑伞 谁在练箭之后 镜里的世界 越过越远的道别 你转身向背 侧脸还是很美 我用眼
